In [84]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import math 

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

SEQ_LEN = 50

In [85]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [86]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [87]:
! pwd
! ls

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		CNN_W2P_model  global.r        server.r
BIG_q.ipynb		data	       make_ARIMA.r    trained
Build_model_task.ipynb	Energy_ts      sarah_data.csv  ui.R


In [88]:
## move the csv file to the bucket ~ you rly dont need to do this
!gsutil cp -p /content/datalab/energy_forcasing/Sarah/sarah_data.csv gs://sarah-bucket/sarah_data.csv

Copying file:///content/datalab/energy_forcasing/Sarah/sarah_data.csv [Content-Type=text/csv]...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      


In [89]:
CSV_COLUMNS = ['prediction_date', 'wind_speed_100m', 
  'wind_direction_100m', 
  'temperature', 
  'air_density',
  'pressure', 'precipitation', 'wind_gust', 'radiation', 
  'wind_speed', 'wind_direction', 'price']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df5 = pd.read_csv('/content/datalab/energy_forcasing/Sarah/sarah_data.csv', header = None, names = CSV_COLUMNS)
df5.describe()

,wind_speed_100m,wind_direction_100m,temperature,air_density,pressure,precipitation,wind_gust,radiation,wind_speed,wind_direction,price
count,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000
mean,4.799901,189.582975,14.920355,1.132338,935.286695,0.117364,4.872509,225.250182,3.220986,189.082629,50.153506
std,1.964154,58.126639,9.626527,0.039333,6.105083,0.245561,2.428587,266.299815,1.319704,56.712796,12.448392
min,1.372222,47.883333,-6.044444,1.036111,913.800000,0.000000,1.194444,0.000000,0.855556,50.538889,4.000000
25%,3.534722,142.598611,7.540278,1.106111,932.034722,0.000000,3.284722,0.000000,2.294444,143.750000,43.000000
50%,4.477778,191.908333,14.213889,1.131667,934.761111,0.000000,4.327778,95.000000,2.961111,191.208333,50.890000
75%,5.544444,232.043056,21.062500,1.160556,937.805556,0.100000,5.627778,416.422222,3.850000,230.865278,59.950000
max,15.044444,329.544444,41.044444,1.229444,961.238889,1.700000,16.705556,954.288889,9.738889,327.711111,85.050000


In [90]:
df5.shape

(6646, 12)

In [91]:
#for ii in range(10):
#   N = 7
#  SEQ = df5.price[(0+N*ii):(N+N*ii)]
np.random.seed(1) 
# np.random.uniform(0,6646-N)

In [92]:
# int(np.random.uniform(0,6646-N),0)

In [93]:
SEQ_LEN = 24 # Sequence of 1 week
with open('./data/timeserie_price_daily_train.csv', 'w') as f:
    to_csv = ''
    for idx, p in enumerate(df5['price']):
        to_csv += str(p) + ','
        if (idx + 1) % SEQ_LEN == 0:
            f.write(to_csv[:-1] + '\n')
            to_csv = ''

In [94]:
#int(round(np.random.uniform(0,6646-N),0))
#6646/50

In [95]:
def to_csv(filename, N, J):
  with open(filename, 'w') as ofp:
    start = 0
    end = J
    for ii in xrange(0, N):
      nn = int(round(np.random.uniform(low = 0, high = 6466-N),0))
      seq = df5.price[(nn):(nn+J)]
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass
to_csv('data/CNN_W2P_model/train-1.csv', N = 1000, J = 50)  # 1000 sequences
to_csv('data/CNN_W2P_model/valid-1.csv', N = 150, J = 50)

In [96]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()
#df2.describe()
# df2.count()
df2.shape
## print(df2.iloc[0])

(1000, 50)

In [97]:
msk = np.random.rand(len(df2)) < 0.8
train = df2[msk]
test = df2[~msk]
train
# then normalize
# then write to train.csv, test.csv df2.to_csv(...)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,55.11,62.00,46.79,44.00,44.00,44.86,60.89,58.69,36.49,57.69,...,42.46,47.02,37.10,38.00,38.00,60.99,50.01,19.52,39.98,47.96
1,50.56,55.67,65.19,62.50,51.10,48.62,49.85,54.11,49.99,62.00,...,33.73,26.20,39.25,49.38,39.10,49.98,30.13,54.82,12.00,52.97
3,53.11,59.69,35.00,56.15,63.10,39.44,45.25,64.19,58.95,57.48,...,34.11,53.91,28.40,48.41,48.77,57.98,45.05,55.00,49.57,47.00
4,27.39,28.46,58.00,31.50,55.83,28.21,67.94,40.81,57.60,54.50,...,63.31,64.11,69.34,56.50,43.42,35.50,47.94,65.69,52.69,55.50
5,67.00,45.10,38.15,43.82,33.49,44.12,37.63,55.68,67.00,65.48,...,57.00,48.62,51.75,49.10,59.75,62.69,51.31,52.37,64.18,69.01


In [98]:
df2 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df2.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,...,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,48.657533,48.547667,49.935533,49.066000,51.845867,48.305533,50.276533,49.329333,51.199000,49.821600,...,51.951000,49.604867,51.193867,49.995000,50.747133,52.083400,50.861933,50.033067,49.769933,49.047000
std,12.825350,13.463538,13.126490,15.322575,11.902066,14.632702,14.682750,14.250683,13.114042,13.361957,...,11.689557,11.972980,11.340560,12.773457,12.543556,10.975638,12.238155,12.772561,11.996131,13.864141
min,5.000000,5.000000,12.000000,4.990000,14.350000,7.000000,4.990000,7.000000,5.000000,4.990000,...,12.700000,4.000000,12.000000,5.000000,5.000000,24.100000,10.280000,4.800000,26.260000,5.020000
25%,40.392500,41.000000,41.505000,39.465000,44.447500,41.602500,43.615000,42.025000,43.532500,42.652500,...,45.425000,43.135000,44.125000,42.935000,43.975000,46.382500,43.425000,43.787500,42.025000,40.197500
50%,50.000000,49.995000,50.875000,51.550000,52.410000,50.000000,53.030000,51.590000,52.630000,50.515000,...,52.050000,51.050000,50.775000,50.590000,51.160000,51.400000,52.455000,50.910000,50.270000,49.600000
75%,59.137500,58.960000,60.212500,60.422500,60.025000,59.095000,60.345000,60.107500,61.327500,59.895000,...,60.077500,58.642500,59.902500,59.962500,59.922500,60.100000,60.565000,59.890000,60.400000,61.025000
max,71.750000,70.690000,70.520000,77.150000,75.000000,70.810000,78.900000,73.600000,75.710000,75.690000,...,74.750000,77.150000,71.690000,72.210000,85.050000,77.150000,75.710000,72.210000,71.150000,73.020000


In [99]:
%%bash
rm -rf /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
mkdir /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
touch /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/__init__.py

In [100]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

TIMESERIES_COL = 'price'
N_OUTPUTS = 1  # in each sequence, 1-49 are features, and 50 is label
SEQ_LEN = None
DEFAULTS = None
N_INPUTS = None


def init(hparams):
    global SEQ_LEN, DEFAULTS, N_INPUTS
    SEQ_LEN = hparams['sequence_length']
    DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
    N_INPUTS = SEQ_LEN - N_OUTPUTS


def linear_model(features, mode, params):
    X = features[TIMESERIES_COL]
    predictions = tf.layers.dense(X, 1, activation=None)
    return predictions


def dnn_model(features, mode, params):
    X = features[TIMESERIES_COL]
    h1 = tf.layers.dense(X, 10, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # linear output: regression
    return predictions


def cnn_model(features, mode, params):
    X = tf.reshape(features[TIMESERIES_COL],
                   [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
    c1 = tf.layers.conv1d(X, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

    c2 = tf.layers.conv1d(p1, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)

    outlen = p2.shape[1] * p2.shape[2]
    c2flat = tf.reshape(p2, [-1, outlen])
    h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
    return predictions


def rnn_model(features, mode, params):
    CELL_SIZE = N_INPUTS // 3  # size of the internal state in each of the cells

    # 1. dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell = tf.nn.rnn_cell.GRUCell(CELL_SIZE)
    outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h1 = tf.layers.dropout(inputs=h1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h2 = tf.layers.dense(dropout_h1, N_INPUTS // 2, activation=tf.nn.relu6)
    h3 = tf.layers.dense(h2, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h3 = tf.layers.dropout(inputs=h3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h4 = tf.layers.dense(dropout_h3, N_INPUTS // 2, activation=tf.nn.relu6)
    h5 = tf.layers.dense(h4, N_INPUTS // 2, activation=tf.nn.relu)
    h6 = tf.layers.dense(h5, N_INPUTS // 2, activation=tf.nn.relu6)
    h7 = tf.layers.dense(h6, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h7 = tf.layers.dropout(inputs=h7, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h8 = tf.layers.dense(dropout_h7, N_INPUTS // 2, activation=tf.nn.relu6)
    h9 = tf.layers.dense(h8, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h9 = tf.layers.dropout(inputs=h9, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h10 = tf.layers.dense(dropout_h9, N_INPUTS // 2, activation=tf.nn.relu6)
    h11 = tf.layers.dense(h10, N_INPUTS // 2, activation=tf.nn.relu)
    h12 = tf.layers.dense(h11, N_INPUTS // 2, activation=tf.nn.relu6)
    
    
    predictions = tf.layers.dense(h12, 1, activation=None)  # (?, 1)
    return predictions


# 2-layer RNN
def rnn2_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'state' is now a tuple containing the final state of each cell layer
    # we use state[1] below to extract the final state of the final layer
    
    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state[1], cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# create N-1 predictions
def rnnN_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'outputs' contains the state of the final layer for every time step
    # not just the last time step (?,N_INPUTS, final cell size)
    
    # 3. pass state for each time step through a DNN, to get a prediction
    # for each time step 
    h1 = tf.layers.dense(outputs, cells.output_size, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # (?, N_INPUTS, 1)
    predictions = tf.reshape(predictions, [-1, N_INPUTS])
    return predictions # return prediction for each time step


# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {TIMESERIES_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn


def serving_input_fn():
    feature_placeholders = {
        TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
    }

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2])

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def compute_errors(features, labels, predictions):
    labels = tf.expand_dims(labels, -1)  # rank 1 -> rank 2 to match rank of predictions

    if predictions.shape[1] == 1:
        loss = tf.losses.mean_squared_error(labels, predictions)
        rmse = tf.metrics.root_mean_squared_error(labels, predictions)
        return loss, rmse
    else:
        # one prediction for every input in sequence
        # get 1-N of (x + label)
        labelsN = tf.concat([features[TIMESERIES_COL], labels], axis=1)
        labelsN = labelsN[:, 1:]
        # loss is computed from the last 1/3 of the series
        N = (2 * N_INPUTS) // 3
        loss = tf.losses.mean_squared_error(labelsN[:, N:], predictions[:, N:])
        # rmse is computed from last prediction and last label
        lastPred = predictions[:, -1]
        rmse = tf.metrics.root_mean_squared_error(labels, lastPred)
        return loss, rmse

# RMSE when predicting same as last value
def same_as_last_benchmark(features, labels):
    predictions = features[TIMESERIES_COL][:,-1] # last value in input sequence
    return tf.metrics.root_mean_squared_error(labels, predictions)


# create the inference model
def sequence_regressor(features, labels, mode, params):
    # 1. run the appropriate model
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'cnn': cnn_model,
        'rnn': rnn_model,
        'rnn2': rnn2_model,
        'rnnN': rnnN_model}
    model_function = model_functions[params['model']]
    predictions = model_function(features, mode, params)

    # 2. loss function, training/eval ops
    loss = None
    train_op = None
    eval_metric_ops = None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss, rmse = compute_errors(features, labels, predictions)

        if mode == tf.estimator.ModeKeys.TRAIN:
            # this is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                # 2b. set up training operation
                train_op = tf.contrib.layers.optimize_loss(
                    loss,
                    tf.train.get_global_step(),
                    learning_rate=params['learning_rate'],
                    optimizer="Adam")

        # 2c. eval metric
        eval_metric_ops = {
            "RMSE": rmse,
            "RMSE_same_as_last": same_as_last_benchmark(features, labels),
        }

    # 3. Create predictions
    if predictions.shape[1] != 1:
        predictions = predictions[:, -1]  # last predicted value
    predictions_dict = {"predicted": predictions}

    # 4. return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs={
            'predictions': tf.estimator.export.PredictOutput(predictions_dict)}
    )


def train_and_evaluate(output_dir, hparams):
    get_train = read_dataset(hparams['train_data_path'],
                             tf.estimator.ModeKeys.TRAIN,
                             hparams['train_batch_size'])
    get_valid = read_dataset(hparams['eval_data_path'],
                             tf.estimator.ModeKeys.EVAL,
                             1000)
    estimator = tf.estimator.Estimator(model_fn=sequence_regressor,
                                       params=hparams,
                                       config=tf.estimator.RunConfig(
                                           save_checkpoints_secs=hparams['min_eval_frequency']),
                                       model_dir=output_dir)
    train_spec = tf.estimator.TrainSpec(input_fn=get_train,
                                        max_steps=hparams['train_steps'])
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_valid,
                                      steps=None,
                                      exporters=exporter,
                                      start_delay_secs=hparams['eval_delay_secs'],
                                      throttle_secs=hparams['min_eval_frequency'])
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Writing /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py


In [101]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example implementation of code to run on the Cloud ML service.
"""

import traceback
import argparse
import json
import os
from . import model

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--train_data_path',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--eval_data_path',
      help='GCS or local path to evaluation data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--learning_rate',
      help='Initial learning rate for training',
      type=float,
      default=0.01
  )
  parser.add_argument(
      '--train_steps',
      help="""\
      Steps to run the training job for. A step is one batch-size,\
      """,
      type=int,
      default=0
  )
  parser.add_argument(
      '--sequence_length',
      help="""\
      This model works with fixed length sequences. 1-(N-1) are inputs, last is output
      """,
      type=int,
      default=10
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  model_names = [name.replace('_model','') \
                   for name in dir(model) \
                     if name.endswith('_model')]
  parser.add_argument(
      '--model',
      help='Type of model. Supported types are {}'.format(model_names),
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=60,
      type=int
  )

  args = parser.parse_args()
  hparams = args.__dict__
  
  # unused args provided by service
  hparams.pop('job_dir', None)
  hparams.pop('job-dir', None)

  output_dir = hparams.pop('output_dir')

  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  output_dir = os.path.join(
      output_dir,
      json.loads(
          os.environ.get('TF_CONFIG', '{}')
      ).get('task', {}).get('trial', '')
  )

  # calculate train_steps if not provided
  if hparams['train_steps'] < 1:
     # 1,000 steps at batch_size of 100
     hparams['train_steps'] = (1000 * 100) // hparams['train_batch_size']
     print ("Training for {} steps".format(hparams['train_steps']))

  model.init(hparams)

  # Run the training job
  model.train_and_evaluate(output_dir, hparams)


Writing /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py


In [102]:
! pwd
! ls
#!{PWD}/CNN_W2P_model/trainer

## /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		CNN_W2P_model  global.r        server.r
BIG_q.ipynb		data	       make_ARIMA.r    trained
Build_model_task.ipynb	Energy_ts      sarah_data.csv  ui.R


## run the model locally on the vm

In [103]:

%%bash

DATADIR=$(pwd)/data/CNN_W2P_model 
OUTDIR=$(pwd)/CNN_W2P_model/trained
SEQ_LEN=50
rm -rf $OUTDIR
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/CNN_W2P_model/trainer \
   -- \
   --train_data_path="${DATADIR}/train-1.csv" \
   --eval_data_path="${DATADIR}/valid-1.csv"  \
   --output_dir=${OUTDIR} \
   --model=rnn --train_steps=10  --sequence_length=$SEQ_LEN

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/train-1.csv', u'--eval_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/valid-1.csv', u'--output_dir=/content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trained', u'--model=rnn', u'--train_steps=10', u'--sequence_length=50'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 60, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <te

In [104]:
import shutil
shutil.rmtree('/data/CNN_W2P_model ', ignore_errors=True)
#os.makedirs('/data/CNN_W2P_model/')
np.random.seed(1) # makes data generation reproducible
for i in xrange(0,10):
  to_csv('data/CNN_W2P_model/train-{}.csv'.format(i), N = 1000, J = 50)  # 1000 sequences
  to_csv('data/CNN_W2P_model/valid-{}.csv'.format(i), N = 1500, J = 50)

In [105]:
%bash
gsutil -m rm -rf gs://${BUCKET}/CNN_W2P_model/*
gsutil -m cp data/CNN_W2P_model/*.csv gs://${BUCKET}/CNN_W2P_model

Removing gs://sarah-bucket/CNN_W2P_model/to_model.csv#1537477772348081...
Removing gs://sarah-bucket/CNN_W2P_model/train-0.csv#1537477772326815...
Removing gs://sarah-bucket/CNN_W2P_model/train-1.csv#1537477772340835...
Removing gs://sarah-bucket/CNN_W2P_model/train-2.csv#1537477772348923...
Removing gs://sarah-bucket/CNN_W2P_model/train-4.csv#1537477772493306...
Removing gs://sarah-bucket/CNN_W2P_model/train-3.csv#1537477772389020...
Removing gs://sarah-bucket/CNN_W2P_model/train-5.csv#1537477772511617...
Removing gs://sarah-bucket/CNN_W2P_model/train-6.csv#1537477772510071...
Removing gs://sarah-bucket/CNN_W2P_model/train-7.csv#1537477772527892...
Removing gs://sarah-bucket/CNN_W2P_model/train-8.csv#1537477772542468...
Removing gs://sarah-bucket/CNN_W2P_model/train-9.csv#1537477772576112...
Removing gs://sarah-bucket/CNN_W2P_model/valid-0.csv#1537477772596422...
Removing gs://sarah-bucket/CNN_W2P_model/valid-1.csv#1537477772648757...
Removing gs://sarah-bucket/CNN_W2P_model/valid-2.c

### Push out to the cloud

In [106]:
%%bash
for MODEL in cnn rnn rnn2 rnnN; do ## linear dnn 
  OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
  JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)
  SEQ_LEN=50
  REGION=us-central1
  gsutil -m rm -rf $OUTDIR
  echo $JOBNAME
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=5000 --sequence_length=$SEQ_LEN --model=$MODEL
done


CNN_W2P_model_cnn180920_213621
jobId: CNN_W2P_model_cnn180920_213621
state: QUEUED
CNN_W2P_model_rnn180920_213624
jobId: CNN_W2P_model_rnn180920_213624
state: QUEUED
CNN_W2P_model_rnn2180920_213628
jobId: CNN_W2P_model_rnn2180920_213628
state: QUEUED
CNN_W2P_model_rnnN180920_213632
jobId: CNN_W2P_model_rnnN180920_213632
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [CNN_W2P_model_cnn180920_213621] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe CNN_W2P_model_cnn180920_213621

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs CNN_W2P_model_cnn180920_213621
CommandException: 1 files/objects could not be removed.
Job [CNN_W2P_model_rnn180920_213624] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe CNN_W2P_model_rnn180920_213624

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs CNN_W2P_model_rnn180920_213624
CommandException: 1 files/objects could not be removed.
Job [CNN_W2P_model_rnn2180920_213628] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe CNN

In [108]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/CNN_W2P_model'.format(BUCKET))

TensorBoard was started successfully with pid 23394. Click here to access it.

23394

In [46]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 15244


In [47]:
## command line access of the TB
print('tensorboard --logdir=gs://{}/CNN_W2P_model --port=8083'.format(BUCKET))

tensorboard --logdir=gs://sarah-bucket/CNN_W2P_model --port=8083


### Deploy the model

In [74]:
%bash
gsutil ls gs://${BUCKET}/CNN_W2P_model/cnn/export/exporter/

gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537478189/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537478248/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537478368/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537478491/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537478610/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537478670/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/temp-1537478792/


In [75]:
%bash
MODEL_NAME="CNN_W2P_model_cnn"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/CNN_W2P_model/cnn/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying CNN_W2P_model_cnn ml_on_gcp from gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537478792/ ... this will take a few minutes


ERROR: (gcloud.ml-engine.models.create) Resource in project [qwiklabs-gcp-aebfb78fe0f1b1d1] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
ERROR: (gcloud.ml-engine.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name


In [76]:
df5.shape[0] 


6646

### Convert the data into something that can be read into the model, so strings that are 49 long where the 50th price is perdicted.

In [77]:
def to_model_csv(filename, N, J):
  with open(filename, 'w') as ofp:
    N = df5.shape[0] - J
    for ii in xrange(0, N):
      seq = df5.price[(0+ii):(J+ii)]
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass

to_model_csv('data/CNN_W2P_model/to_model.csv', N = 1000, J = 49) 


In [78]:
model_data = pd.read_csv('/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/to_model.csv', header = None)
model_data.shape

(6597, 49)

In [79]:
#prices = model_data.iloc[0]
#string(prices)
raw_prices = model_data.iloc[0]

#raw_prices = "55.11,62.0,46.79,44.0,44.0,44.86,60.89,58.69,36.49,57.69,51.2,\
#48.79,48.35,42.2,46.37,62.5,62.5,54.0,52.54,62.18,48.5,55.35,52.0,40.97,42.6,64.1,\
#61.59,48.9,37.4,45.93,37.3,65.48,62.5,65.03,57.07,23.35,54.56,28.02,39.84,51.0,42.46,47.02,37.1,38.0,38.0,60.99,50.01,19.52,39.98,47.96".split(",")

prices = [float(price) for price in raw_prices]
#model_data.iloc[0]

In [80]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME="CNN_W2P_model_cnn"
MODEL_VERSION="ml_on_gcp"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'price': prices
    },
  ]
}

In [81]:
response = requests.post(api, json=data, headers=headers)
print response.content

{"predictions": [{"predicted": [50.595191955566406]}]}


In [82]:
##55.11,62.0,46.79,44.0,44.0,44.86,60.89,58.69,36.49,57.69,51.2,48.79,48.35,42.2,46.37,62.5,62.5,54.0,52.54,62.18,48.5,55.35,52.0,40.97,42.6,64.1,61.59,48.9,37.4,45.93,37.3,65.48,62.5,65.03,57.07,23.35,54.56,28.02,39.84,51.0,42.46,47.02,37.1,38.0,38.0,60.99,50.01,19.52,39.98,47.96

This will take a few minutes.

In [83]:
result = []
for y in range(1,5):#model_data.shape[0]):
    raw_prices = model_data.iloc[y]
    prices = [float(price) for price in raw_prices]
    print prices
    data = {
      'instances': [
        {
          'price': prices
        },
      ]
    }
    response = requests.post(api, json=data, headers=headers)
    print y/model_data.shape[0]
    print response.content[32:-4]
    result.append(response.content[32:-4])

[56.55, 54.74, 30.0, 46.86, 39.25, 48.5, 43.49, 59.57, 40.13, 45.0, 49.85, 50.07, 49.0, 43.69, 63.03, 45.59, 53.56, 38.12, 55.89, 40.16, 65.1, 39.58, 59.85, 55.0, 64.19, 53.28, 68.6, 41.0, 39.25, 38.35, 55.76, 40.32, 62.59, 46.78, 49.85, 59.69, 56.0, 65.1, 58.69, 39.5, 30.0, 65.1, 57.2, 30.0, 50.76, 47.08, 45.1, 14.84, 60.1]
0
50.595191955566406
[54.74, 30.0, 46.86, 39.25, 48.5, 43.49, 59.57, 40.13, 45.0, 49.85, 50.07, 49.0, 43.69, 63.03, 45.59, 53.56, 38.12, 55.89, 40.16, 65.1, 39.58, 59.85, 55.0, 64.19, 53.28, 68.6, 41.0, 39.25, 38.35, 55.76, 40.32, 62.59, 46.78, 49.85, 59.69, 56.0, 65.1, 58.69, 39.5, 30.0, 65.1, 57.2, 30.0, 50.76, 47.08, 45.1, 14.84, 60.1, 60.5]
0
50.595191955566406
[30.0, 46.86, 39.25, 48.5, 43.49, 59.57, 40.13, 45.0, 49.85, 50.07, 49.0, 43.69, 63.03, 45.59, 53.56, 38.12, 55.89, 40.16, 65.1, 39.58, 59.85, 55.0, 64.19, 53.28, 68.6, 41.0, 39.25, 38.35, 55.76, 40.32, 62.59, 46.78, 49.85, 59.69, 56.0, 65.1, 58.69, 39.5, 30.0, 65.1, 57.2, 30.0, 50.76, 47.08, 45.1, 14.84

In [58]:
print result

['50.6190299987793', '50.6190299987793', '50.6190299987793', '50.6190299987793']


In [59]:
print response.content[32:-4]

50.6190299987793
